In [ ]:
import json

# 读取JSON文件
with open('transcription_qna_2500.json', 'r') as file:
    json_data = json.load(file)

# 初始化变量
qa_pairs = []
video_ids_set = set()  # 使用集合避免重复
video_ids = []  # 按顺序记录视频ID
qa_pair_count = 0  # 记录已提取的QA对数量

# 遍历每个子字典
for entry in json_data:
    # 每个子字典的 "firestore_data" 中的视频ID
    videos = entry.get("firestore_data", {}).get("videos", [])
    
    # 获取该entry中的QA对
    qna_formats = entry.get("qna_format", [])
    
    # 遍历每个qna格式，提取QA对
    for qna_format in qna_formats:
        conversations = qna_format.get("conversation", [])
        
        for conversation in conversations:
            # 统计QA对
            qa_pairs.append({
                'input': conversation['input'],
                'output': conversation['output']
            })
            qa_pair_count += 1  # 计数QA对

            # 如果这个视频ID不在集合中，加入集合和列表
            if videos:
                video_id = videos[0]  # 每个entry只对应一个视频ID
                if video_id not in video_ids_set:
                    video_ids_set.add(video_id)
                    video_ids.append(video_id)

            # 如果达到2000个QA对，停止提取
            if qa_pair_count >= 2000:
                break
        if qa_pair_count >= 2000:
            break
    if qa_pair_count >= 2000:
        break
print(f"提取的QA对数量: {len(qa_pairs)}")
print(f"提取的视频ID数量: {len(video_ids)}")
# ffff
###########################################################average pooling and gif convert###############################################################
# import cv2
# import numpy as np
# from PIL import Image
# import os

# # 确保保存目录存在
# os.makedirs("images_fromvideo", exist_ok=True)

# # 平均池化处理视频，生成图像
# def average_pooling_image(video_path):
#     cap = cv2.VideoCapture(video_path)
#     frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     # 如果视频没有帧，返回空
#     if frame_count == 0:
#         print(f"视频 {video_path} 没有帧，跳过处理。")
#         return None

#     # 初始化图像数组
#     avg_frame = np.zeros((height, width, 3), np.float32)

#     for _ in range(frame_count):
#         ret, frame = cap.read()
#         if not ret:
#             break
#         avg_frame += frame

#     avg_frame /= frame_count
#     avg_frame = np.array(np.round(avg_frame), dtype=np.uint8)

#     # 返回生成的平均池化图像
#     cap.release()
#     return Image.fromarray(cv2.cvtColor(avg_frame, cv2.COLOR_BGR2RGB))

# # 转换视频为GIF
# def convert_video_to_gif(video_path, output_path):
#     cap = cv2.VideoCapture(video_path)
#     frames = []
#     success, image = cap.read()
    
#     # 如果视频没有帧，跳过处理
#     if not success:
#         print(f"无法读取视频 {video_path}")
#         return
    
#     while success:
#         img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#         frames.append(img)
#         success, image = cap.read()

#     # 保存为动图（GIF）
#     if frames:
#         frames[0].save(output_path, save_all=True, append_images=frames[1:], duration=100, loop=0)
#     cap.release()

# # # 假设你已经获得了5个视频的ID
# # video_ids = ["ZCkgO3lsnek8vB1JbofL", "X0VKaeA8rasmGenaosfl", "Lh8DVZuI734lnc95fnJw", "其他视频ID", "其他视频ID"]

# # 批量处理视频
# for video_id in video_ids:
#     video_path = f"downloaded_videos_2000/{video_id}.mp4"  # 假设视频存储在 "videos" 目录下，并且文件名为 {video_id}.mp4
    
#     # 生成平均池化图像
#     average_image = average_pooling_image(video_path)
#     if average_image:
#         average_image.save(f"images_fromvideo/{video_id}_average.png")
#         print(f"平均池化图像已保存: images_fromvideo/{video_id}_average.png")

#     # 生成GIF
#     gif_path = f"images_fromvideo/{video_id}.gif"
#     convert_video_to_gif(video_path, gif_path)
#     print(f"GIF 已保存: {gif_path}")
##############################################################################################################################################################

import os
import imageio_ffmpeg as ffmpeg
import subprocess
import json
import requests
import os

# 确保保存目录存在
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# 使用 imageio-ffmpeg 从视频中提取帧
def extract_frames_from_video_ffmpeg(video_path, video_id, output_folder, fps=2):
    """
    使用 imageio-ffmpeg 从短视频中提取帧并保存为图像
    video_path: 输入的视频文件路径
    video_id: 视频ID，用于命名图像文件
    output_folder: 图像保存的目录
    fps: 每秒提取帧的数量，默认为每秒2帧
    """
    # 创建保存图像的文件夹
    ensure_dir(output_folder)
    
    # 获取 ffmpeg 可执行文件路径
    ffmpeg_path = ffmpeg.get_ffmpeg_exe()
    
    # 构建 FFmpeg 命令
    output_pattern = os.path.join(output_folder, f"{video_id}_frame_%04d.jpg")
    ffmpeg_command = [
        ffmpeg_path, "-i", video_path,
        "-vf", f"fps={fps}",  # 每秒提取 fps 帧
        output_pattern
    ]
    
    # 执行 FFmpeg 命令
    try:
        subprocess.run(ffmpeg_command, check=True)
        print(f"视频 {video_id} 提取完成并保存到 {output_folder}")
    except subprocess.CalledProcessError as e:
        print(f"提取视频 {video_id} 失败: {str(e)}")


# 批量处理多个视频
def process_videos(video_ids, video_folder, output_folder, fps=3):
    """
    video_ids: 视频ID列表
    video_folder: 视频所在文件夹
    output_folder: 保存图像的文件夹
    fps: 每秒提取多少帧图像，默认3帧
    """
    for video_id in video_ids:
        video_path = os.path.join(video_folder, f"{video_id}.mp4")
        if os.path.exists(video_path):
            print(f"正在处理视频: {video_id}")
            extract_frames_from_video_ffmpeg(video_path, video_id, output_folder, fps)
        else:
            video_url_api = "https://us-central1-sportsbox-3dgolf.cloudfunctions.net/api/videos/url"
            headers = {
                "Authorization": "aFwkNaGuqvEIHtP-M6LQDTNevSF=XfPMJm-GxVT/osOMZwn8HIZEZGH?64S7IwPB",  # 使用Authorization而非x-api-key
                "Content-Type": "application/json"
            }
            # 将视频ID列表分批次请求（假设API有请求数量限制）
            batch_size = 1  # 根据API限制设置批次大小
            video_urls = {}

            for i in range(0, len(video_id)):
                batch_video_ids = video_id
                payload = {
                    "videoIds": batch_video_ids
                }

                # 发送POST请求获取下载链接
                response = requests.post(video_url_api, json=payload, headers=headers)

                # 检查请求是否成功
                if response.status_code == 200:
                    try:
                        video_url_data = response.json()
                        if not video_url_data:
                            print("未收到任何视频数据，返回结果为空")
                            continue

                        # 提取每个视频的下载链接并存储在字典中（key为video_id，value为下载链接）
                        for video in video_url_data:
                            path = video.get('path')
                            video_id = video.get('id')
                            if path and video_id:
                                video_urls[video_id] = path
                        print(f"成功获取 {len(video_url_data)} 个视频下载链接")
                    except ValueError as e:
                        print(f"解析JSON时出错: {e}")
                        continue
                else:
                    print(f"请求失败，状态码: {response.status_code}")
                    continue

            # 打印总共获取的视频下载链接数量
            print(f"总共获取到的视频下载链接数量: {len(video_urls)}")

            # 创建保存视频的目录
            os.makedirs("downloaded_videos_2000", exist_ok=True)

            # 下载每个视频并保存
            for video_id, video_url in video_urls.items():
                try:
                    # 发送GET请求下载视频
                    response = requests.get(video_url, stream=True)

                    # 检查是否成功获取视频数据
                    if response.status_code == 200:
                        # 构建保存路径，使用video_id作为文件名
                        file_path = os.path.join("downloaded_videos_2000", f"{video_id}.mp4")

                        # 以二进制写入模式保存视频
                        with open(file_path, "wb") as video_file:
                            for chunk in response.iter_content(chunk_size=8192):  # 按块写入
                                if chunk:
                                    video_file.write(chunk)

                        print(f"视频 {video_id} 已成功下载并保存到 {file_path}")
                    else:
                        print(f"下载视频 {video_id} 失败，状态码: {response.status_code}")
                except Exception as e:
                    print(f"下载视频 {video_id} 时出现错误: {e}")
            extract_frames_from_video_ffmpeg(video_path, video_id, output_folder, fps)


# 示例视频ID列表
# video_ids = ['ZCkgO3lsnek8vB1JbofL', 'X0VKaeA8rasmGenaosfl', 'Lh8DVZuI734lnc95fnJw']  # 替换为你的视频ID
video_folder = "downloaded_videos_2000"  # 视频文件所在的文件夹
output_folder = "extracted_images_2000"  # 保存图像的文件夹
fps = 6  # 每秒提取3帧

process_videos(video_ids, video_folder, output_folder, fps)

